In [30]:
import torch
from transformers import AutoTokenizer, AutoModel
import onnx
import onnxruntime
from pathlib import Path
import os


In [31]:
# 현재 작업 디렉토리 기준으로 상대 경로 설정
model_path = "../AI/embedding/dragonkue/multilingual-e5-small-ko"
onnx_path = "../AI/embedding/onnx/multilingual-e5-small-ko.onnx"

In [32]:
# 모델과 토크나이저 로드 (로컬 경로 사용)
tokenizer = AutoTokenizer.from_pretrained(str(model_path) , use_fast=True)
model = AutoModel.from_pretrained(str(model_path))
print(f"로컬 모델을 로드했습니다: {model_path}")

로컬 모델을 로드했습니다: ../AI/embedding/dragonkue/multilingual-e5-small-ko


In [14]:
# 모델을 평가 모드로 설정
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(250037, 384, padding_idx=0)
    (position_embeddings): Embedding(512, 384)
    (token_type_embeddings): Embedding(2, 384)
    (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=384, out_features=384, bias=True)
            (key): Linear(in_features=384, out_features=384, bias=True)
            (value): Linear(in_features=384, out_features=384, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=384, out_features=384, bias=True)
            (LayerNorm): LayerNorm((384,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=Fals

In [15]:
# 더미 입력 생성 (배치 크기 1, 시퀀스 길이 512)
dummy_input = torch.randint(0, tokenizer.vocab_size, (1, 512))

In [21]:
# ONNX 모델 변환
# PyTorch 모델을 ONNX 형식으로 변환
# - model: 변환할 PyTorch 모델
# - dummy_input: 모델 입력 형태를 정의하는 더미 데이터
# - onnx_path: 저장할 ONNX 파일 경로
torch.onnx.export(
    model,                          # 변환할 PyTorch 모델
    dummy_input,                    # 모델 입력 형태 정의용 더미 데이터
    onnx_path,                      # ONNX 파일 저장 경로
    export_params=True,             # 모델 파라미터를 ONNX 파일에 포함
    opset_version=14,               # ONNX 연산자 세트 버전 (14로 변경하여 scaled_dot_product_attention 지원)
    do_constant_folding=True,       # 상수 폴딩 최적화 활성화 (추론 속도 향상)
    input_names=['input_ids'],      # 입력 텐서의 이름 지정
    output_names=['last_hidden_state'],  # 출력 텐서의 이름 지정
    dynamic_axes={                  # 동적 축 설정 (배치 크기와 시퀀스 길이를 가변으로 설정)
        'input_ids': {0: 'batch_size', 1: 'sequence_length'},           # 입력: 배치 크기와 시퀀스 길이 가변
        'last_hidden_state': {0: 'batch_size', 1: 'sequence_length'}    # 출력: 배치 크기와 시퀀스 길이 가변
    }
)

print(f"ONNX 모델이 저장되었습니다: {onnx_path}")

ONNX 모델이 저장되었습니다: ../AI/embedding/onnx/multilingual-e5-small-ko.onnx


In [ ]:
# optimum 라이브러리를 사용한 ONNX 변환 (추천 방법)
from optimum.onnxruntime import ORTModelForFeatureExtraction
from pathlib import Path

# 출력 디렉토리 생성
onnx_dir = Path("../AI/embedding/onnx/multilingual-e5-small-ko")
onnx_dir.mkdir(parents=True, exist_ok=True)

# ONNX 모델 변환 및 저장
ort_model = ORTModelForFeatureExtraction.from_pretrained(
    model_path, 
    export=True,  # ONNX 변환 활성화
    use_cache=False,  # 캐시 사용 안함
    file_name="model.onnx",
    provider="CPUExecutionProvider"  # CPU 실행 제공자
)

# 모델과 토크나이저를 지정된 디렉토리에 저장
ort_model.save_pretrained(onnx_dir)
tokenizer.save_pretrained(onnx_dir)

print(f"ONNX 모델이 저장되었습니다: {onnx_dir}")
print(f"모델 파일: {onnx_dir / 'model.onnx'}")
print(f"토크나이저도 함께 저장되었습니다.")


In [22]:
# ONNX 모델 검증
onnx_model = onnx.load(onnx_path)
onnx.checker.check_model(onnx_model)
print("ONNX 모델 검증 완료")

ONNX 모델 검증 완료


In [23]:
# ONNX Runtime으로 추론 테스트
ort_session = onnxruntime.InferenceSession(str(onnx_path))
ort_inputs = {ort_session.get_inputs()[0].name: dummy_input.numpy()}
ort_outputs = ort_session.run(None, ort_inputs)
print("ONNX Runtime 추론 테스트 완료")

ONNX Runtime 추론 테스트 완료
